# Machine learning on mobile
<img src="files/catvsdog.png" />
The goal is to get a trained image classifier working on your phone. The image classifier should be able to detect if an image is a cat or a dog and what percentage cat or dog the picture is.

Steps needed to complete this project.

1. Gather data for training a machine learning model and clean up the data.
2. Traing an image classifier using Keras.
3. Create a restful API that we can send a request to that uses our trained model.
4. Build a mobile application that can take a picture and classify how much cat or dog a picture is

This tutorial is inspired by this Keras example.
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

Here is an image of the entire machine learning data process.
<img src="files/datasteps.png" />


# Gather data for training a machine learning model
<img src="files/puppy.jpg" />
In this tutorial, we will present a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples --just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:

* how to structure training and test data
* training a small network from scratch (as a baseline) on image data
* how to improve model predictions

To acquire a few hundreds or thousands of training images belonging to the classes you are interested in, one possibility would be to use the Flickr API to download pictures matching a given tag, under a friendly license.

In our examples we will use two sets of pictures, which we got from Kaggle: 1000 cats and 1000 dogs (although the original dataset had 12,500 cats and 12,500 dogs, we just took the first 1000 images for each class). We also use 400 additional samples from each class as validation data, to evaluate our models.

That is very few examples to learn from, for a classification problem that is far from simple. So this is a challenging machine learning problem, but it is also a realistic one: in a lot of real-world use cases, even small-scale data collection can be extremely expensive or sometimes near-impossible (e.g. in medical imaging). Being able to make the most out of very little data is a key skill of a competent data scientist.

1. The data can be downloaded from kaggle. Because we are awesome we also stored the data on our floydhob server
https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/

https://www.kaggle.com/c/dogs-vs-cats
    
 
Step 1.
Download the data from kaggle and sort the data into two different folders
Train containing 1000 cat images and 1000 dog images and Validation containing 400 images

Here is how you should structure your folders.
<img src="files/folderstructure.png" />


## Preprocessing
The greatest part of machine learning is fitting your data into the right format. Luckily for you we have already helped you out with this. However it is good to know that getting your data in the right format is a huge part of machine learning

# Convolutional neural networks
<img src="files/visualcortex.png" />
<img src="files/convolution.jpg" />

Convolutional neural networks are neural network that are specialised in image classification. Their functioning is inspired by the mammalian visual cortex.

In [1]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

# flowdhub folder
flowdhub = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/'
flowhubtrain = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/train'
flowdhubvalidation = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/validation'

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 20
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 2002 images belonging to 4 classes.
Found 801 images belonging to 2 classes.
Epoch 1/20
125/125 [==============================] - 49s 396ms/step - loss: 0.7337 - acc: 0.5170 - val_loss: 0.7083 - val_acc: 0.5413
Epoch 2/20
 14/125 [==>...........................] - ETA: 36s - loss: 0.6871 - acc: 0.5714

KeyboardInterrupt: 

In [5]:
# Saving the model weights
model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

In [6]:
# Loading the saved model
from keras.models import model_from_json
# Load trained model
# load json and create model
json_file = open('first_try.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
global loaded_model
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("first_try.h5")
print("Loaded model from disk")

Loaded model from disk


In [1]:
# Reshaping our trained image
import scipy.misc

testimage = scipy.misc.imresize(scipy.misc.imread('data/validation/cats/cat.1211.jpg'), (150, 150))
testimage = testimage.reshape((1,) + testimage.shape)

# checking the shape of our new image
testimage.shape

(1, 150, 150, 3)

# # Lets test our prediction on a test image
<img src="data/validation/cats/cat.1211.jpg" />

In [8]:
prediction = loaded_model.predict(testimage)
print(prediction)

if (prediction[0] == 1):
        print("Cat")
        one = True
else:
    print("Dog")


[[ 1.]]
Cat


# Our classification works but it isn't that good.
What can we do to improve our predictions? Hint. You could probably change settings for how you train your model.

# Trained keras model
To be able to serve a model we need to train a model 
that can be returned with an API call. We have already trained our model. So now our job is to load our trained model. 

Then we will build an API that returns neural networks best guess of what the image is depicting, a cat or a dog.
